# Generate collaboration metrics

In [1]:
import pandas as pd
from pathlib import Path, PurePath
import os
import sys
import numpy as np

In [2]:
from github2pandas_manager.config_parser import YAML_RequestDefinition
import aux

In [3]:
sys.path.append(os.path.join(sys.path[0],'..','..','FeedbackGeneration'))
from DataHandler import DataHandler

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
result_folder = "./example/notebooks/"
result_file_name = "F_CollaborationRatio"

In [6]:
project_folder_name = "../"
project_parameter_file = "config.yml"

In [7]:
# Just for local testing purposes 
# project_folder_name = "./example/"

In [8]:
request_params = YAML_RequestDefinition(project_folder_name+project_parameter_file)
print(request_params.parameters.project_folder)

./example/data/TaskExamples


In [9]:
path = PurePath(os.getcwd())
if path.name == "notebooks":
    print(os.getcwd())
    os.chdir("../../")
    print(os.getcwd())

/home/zug/Desktop/github2pandas/GitHubClassroomTutorFeedback/example/notebooks
/home/zug/Desktop/github2pandas/GitHubClassroomTutorFeedback


In [10]:
dh = DataHandler(request_params.parameters.project_folder)

example/data/TaskExamples


In [11]:
dh.raw_pdCommits['team'] =  dh.raw_pdCommits['repo_name'].apply(aux.getTeamName)
dh.raw_pdEdits['team'] =  dh.raw_pdEdits['repo_name'].apply(aux.getTeamName)
dh.raw_pdIssues['team'] =  dh.raw_pdIssues['repo_name'].apply(aux.getTeamName)
dh.raw_pdRepositories['team'] =  dh.raw_pdRepositories['repo_name'].apply(aux.getTeamName)
dh.raw_pdUsers['team'] =  dh.raw_pdUsers['repo_name'].apply(aux.getTeamName)

In [12]:
github_users_black_list = ["github-classroom[bot]", "actions-user", "web-flow", "github-actions[bot]", "GitHubAction"]
github_users_black_list_ids = dh.raw_pdUsers [dh.raw_pdUsers.login.isin(github_users_black_list)].anonym_uuid.unique().tolist() 

tutor_black_list = ["SebastianZug", "andre-dietrich", "JayTee42", "galinarudollf", "bpaulwitz",
                    "Sebastian Zug", "gjaeger1", "Florian Richter", "RiFlorian", "JayTee"]
tutor_black_list_ids = dh.raw_pdUsers [dh.raw_pdUsers.login.isin(tutor_black_list)].anonym_uuid.unique().tolist() 

black_list_ids = tutor_black_list_ids + github_users_black_list_ids
black_list_ids

['can-expect-late-month', 'would-hot-power-president']

In [13]:
dh.raw_pdUsers['role'] = "student"
dh.raw_pdUsers.loc[dh.raw_pdUsers.anonym_uuid.isin(github_users_black_list_ids), "role"] = 'github'
dh.raw_pdUsers.loc[dh.raw_pdUsers.anonym_uuid.isin(tutor_black_list_ids), "role"] = 'tutor'

In [14]:
df = pd.merge(dh.raw_pdEdits, dh.raw_pdCommits[['commit_sha', 'author']], how='left', left_on=['commit_sha'], right_on = ['commit_sha'])
df = pd.merge(df, dh.raw_pdUsers[['anonym_uuid', 'role']], how='left', left_on=['author'], right_on = ['anonym_uuid'])

In [34]:
df_files = df[df.role != "github"].groupby(['team', 'author']).filename.unique().to_frame()
df_files_separated = pd.DataFrame(df_files.to_records(),
                                  columns=df_files.index.names + list(df_files.columns))

teams = df_files_separated.groupby('team').author.unique()
dict_teams =teams.to_dict()

In [39]:
count = 0
authors_list =  ['author_A', 'author_B']
df['authorID'] = "Orga"
for index, row in df[df.role != "github"].iterrows():
    if row.team in dict_teams:
        members = dict_teams[row.team]
        if row.author in members:
            idx = list(members).index(row.author)
            df.at[index, 'authorID'] = authors_list[idx]

pdResults = df.groupby(['team', 'authorID']).filename.unique().unstack()
pdResults

authorID,Orga,author_A,author_B
team,,,
42,"[README.md, team.config]",[team.config],NaN
aufgabe3,"[README.md, team.config]","[UML-Diagramm vorher, Pet-administration.cs]",NaN
computerministers,"[README.md, team.config]",NaN,NaN
ichhabkeinteam,"[team.config, README.md, .gitignore, Bunny.cs,...","[team.config, Pet.cs, App, App.dll, App.csproj...",NaN
irgendwas,"[README.md, team.config]","[IStokeable.cs, pet.cs, cat.cs, mammal.cs, rab...","[Aufg6_Fisch.cs, Aufg6_Goldfisch.cs, Aufg6_Pet..."
linux,"[README.md, team.config]",NaN,NaN
nobodyexpectsthespanishinquisition,"[Program.cs, .gitignore, softwareentwicklung_a...",NaN,NaN
plantpoweredprogrammers,"[README.md, team.config]","[mammal.cs, cat.cs, rabbit.cs, pet.cs]","[goldfish.cs, fish.cs, guppy.cs, main.cs, kang..."
poorgrammer,"[README.md, team.config, ClassDiagram.puml, Ma...","[ClassDiagram.puml, team.config, Fish.cs, Gold...",[team.config]


In [40]:
pdResults['CollaborationRatio'] = 0
for index, row in pdResults.iterrows():
    if (not isinstance(row.author_A,float)):
        A_files = list(row.author_A)
    else:
        A_files = []
    if (not isinstance(row.author_B,float)):
        B_files = list(row.author_B)
    else:
        B_files = []
    
    if A_files and B_files:
        all_files = set(A_files) | set(B_files)
    if A_files and not B_files:
        all_files = set(A_files)
    if not A_files and B_files:
        all_files = set(B_files)

    pdResults.at[index, 'CollaborationRatio'] = len(set(A_files) & set(B_files)) / len(all_files)

In [41]:
pdResults.reset_index(inplace=True)
pdResults['team'] = "`" + pdResults['team'] + "`" 
pdResults

authorID,team,Orga,author_A,author_B,CollaborationRatio
0,`42`,"[README.md, team.config]",[team.config],NaN,0.000000
1,`aufgabe3`,"[README.md, team.config]","[UML-Diagramm vorher, Pet-administration.cs]",NaN,0.000000
2,`computerministers`,"[README.md, team.config]",NaN,NaN,0.000000
3,`ichhabkeinteam`,"[team.config, README.md, .gitignore, Bunny.cs,...","[team.config, Pet.cs, App, App.dll, App.csproj...",NaN,0.000000
4,`irgendwas`,"[README.md, team.config]","[IStokeable.cs, pet.cs, cat.cs, mammal.cs, rab...","[Aufg6_Fisch.cs, Aufg6_Goldfisch.cs, Aufg6_Pet...",0.000000
5,`linux`,"[README.md, team.config]",NaN,NaN,0.000000
6,`nobodyexpectsthespanishinquisition`,"[Program.cs, .gitignore, softwareentwicklung_a...",NaN,NaN,0.000000
7,`plantpoweredprogrammers`,"[README.md, team.config]","[mammal.cs, cat.cs, rabbit.cs, pet.cs]","[goldfish.cs, fish.cs, guppy.cs, main.cs, kang...",0.000000
8,`poorgrammer`,"[README.md, team.config, ClassDiagram.puml, Ma...","[ClassDiagram.puml, team.config, Fish.cs, Gold...",[team.config],0.100000
9,`suppengrun`,"[README.md, team.config]","[team.config, UML_code.txt]","[team.config, .gitignore, Program.cs, .NETCore...",0.090909


In [42]:
with open(result_folder + result_file_name + '.txt', 'w') as f:
    f.write(pdResults[['team', 'author_A', 'author_B', 'CollaborationRatio']].to_markdown())